In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import networkx as nx

In [ ]:
cb1 = pd.read_csv('cb1.csv')
cb2 = pd.read_csv('cb2.csv')
cb3 = pd.read_csv('cb3.csv')
cb = cb1.append(cb2)
cb = cb.append(cb3)
cb.index = range(len(cb))
cb['date_my'] = (pd.to_datetime(cb['date'],format='%m/%d/%Y', errors='coerce')).dt.to_period('m')
cb['date_y'] = (pd.to_datetime(cb['date'],format='%m/%d/%Y', errors='coerce')).dt.to_period('y')

nodes_org = cb[['target_name', 'target_ID', 'target_country', 'target_continent','industry']].drop_duplicates()
nodes_org['bipartite'] = 1
nodes_inv = cb[['investors_name', 'investor_id', 'investor_country', 'investor_continent', 'investor_type']].drop_duplicates()
nodes_org = nodes_org.rename(columns={"target_name": "node", "target_country": "country", 
                                      "target_continent": "continent", "industry":"industry",
                                     "target_ID":"index"})
nodes_inv = nodes_inv.rename(columns={"investors_name": "node","investor_country": "country", 
                                      "investor_continent": "continent", "investor_type":"industry","investor_id":'index' })
nodes_inv['bipartite'] = 0
nodes = nodes_org.append(nodes_inv, ignore_index= True)
nodes['id'] = nodes.index

numbers = cb.groupby(['target_name','stage'], as_index = False).count()[['target_name','stage','investors_name']].rename(columns={"investors_name": "investor_numbers"})
cb_new = pd.merge(cb, numbers,  how='left', left_on=['target_name','stage'], right_on = ['target_name','stage'])
cb_new = pd.merge(cb_new, nodes[nodes['bipartite']== 1],  how='left', left_on=['target_name','target_ID'], right_on = ['node','index'])
cb_new = pd.merge(cb_new, nodes[nodes['bipartite']== 0],  how='left', left_on=['investors_name','investor_id'], right_on = ['node','index'])
nodes = nodes.drop(columns = "index")


In [ ]:
cb_new['target_founded']

In [ ]:
nodes.to_csv('nodes.csv', index = False)

In [ ]:
nodes_fda = pd.read_csv('traject_3y.csv')
#covariates = pd.read_csv(r'C:\Users\Marco\Documents\GitHub\crunch_net\ANS\ANS_all\Covariate_bidbid_final_final2.csv')
covariates_org = pd.read_csv(r'C:\Users\Marco\Documents\GitHub\crunch_net\ANS\ANS_all\Covariate_orgorg5.csv')

In [ ]:
orgs = list(nodes_fda.columns)
orgs = [int(x) for x in orgs]

In [ ]:
centralities = list(covariates.columns)
for i in range(0,len(centralities)):
    centralities[i] = centralities[i][:-4]
centralities = list(set(centralities))[1:]

In [ ]:
centralities_org = list(covariates_org.columns)
for i in range(0,len(centralities_org)):
    centralities_org[i] = centralities_org[i][:-4]
centralities_org = list(set(centralities_org))[1:]

In [ ]:
covariates_org['id'] = covariates_org['Unnamed: 0']

In [ ]:
#CENTRALITITES OF ALL

sub0 = cb_new[['id_x','date_y','id_y', 'size_real']]
#for cen in centralities: 
#    locals()[cen+'_max'] = []
#    locals()[cen+'_min'] = []
#    locals()[cen+'_median'] = []
#operators = ['_max', '_min', '_median']  
#num_inv = []

s = 0

for cen_org in centralities_org:
    locals()[cen_org +'_org'] = []

for  i in tqdm(orgs):
    
    sub1 = sub0[sub0['id_x']==i]
    sub1 = sub1.dropna()
    sub2 = sub1[sub1['date_y']==sub1['date_y'].values[0]]
    anno = sub2['date_y'].values[0].year
    

    for cen_org in centralities_org:
        try:
            locals()[cen_org + '_org'].append((covariates_org[covariates_org['id']== i][cen_org + str(anno)].values[0]))
        except:
            locals()[cen_org + '_org'].append(np.nan)
        
    s += 1
    if s % 500 == 0:
        print(s)

In [ ]:
covariate5= pd.DataFrame(nodes[(nodes['id'].isin(orgs))&(nodes['bipartite']==1)]['id'])


for cen_org in centralities_org:
    covariate5[cen_org + '_org'] = locals()[cen_org + '_org']


In [140]:
covariates = pd.read_csv('covariates.csv')
cov_bid = covariates.iloc[:,1:44]
covariates10 =pd.merge(covariate10, cov_bid, left_on='id', right_on='id')
covariates10.to_csv('covariates10.csv')

In [ ]:
a = nx.readwrite.read_gpickle(r'C:\Users\Marco\Documents\GitHub\crunch_net\ANS\ANS_all\orgorg6\orgorg6_single2020.gpickle')
b = nx.readwrite.read_gpickle(r'C:\Users\Marco\Documents\GitHub\crunch_net\ANS\ANS_all\orgorg5\orgorg5_single2020.gpickle')

In [ ]:
c = nx.readwrite.read_gpickle(r'C:\Users\Marco\Documents\GitHub\crunch_net\ANS\ANS_all\orgorg4\orgorg4_single2020.gpickle')

In [ ]:
nx.info(a)

In [ ]:
nx.info(b)

In [ ]:
nx.info(c)

In [ ]:
industries = list(set(covariate['industry']))
settori = {'industry' : industries, 
           'sector' : ['Healthcare', 'Healthcare', 'Healthcare', 'Healthcare','Healthcare','Healthcare','Healthcare','Healthcare','Healthcare', 'Consumer Products & Services', 'Healthcare', 'Software (non-internet/mobile)',
                                     'Internet', 'Healthcare','Healthcare','Healthcare','Healthcare','Healthcare', 'Mobile & Telecommunications', 'Healthcare','Healthcare','Healthcare']
}

settori = pd.DataFrame(settori)

foundation = cb_new[['id_x', 'target_founded']].drop_duplicates()

covariate2 =pd.merge(covariate, foundation, left_on='id', right_on='id_x')
covariate2 =pd.merge(covariate2, settori, left_on='industry', right_on='industry')

covariate2 = covariate2.drop('id_x', axis =1)

foundation = covariate2.pop('target_founded')
sector = covariate2.pop('sector')

covariate2.insert(4, 'foundation', foundation)
covariate2.insert(5, 'sector', sector)

In [ ]:
sub0 = cb_new[['id_x','date_y','round_simp','size_real','target_status']].drop_duplicates()

ids = covariate2['id']
stage = []
first_money = []
date = []
situa = []

for i in tqdm(ids):
    sub1 = sub0[sub0['id_x'] == i]
    sub1 = sub1.dropna()
    stage.append(sub1['round_simp'].values[0])
    first_money.append(sub1['size_real'].values[0])
    situa.append(sub1['target_status'].values[0])
    date.append(sub1['date_y'].values[0])
    
    
stag = {'id' : list(ids),
       'stage' : stage,
       'first_money': first_money,
       'date' : date,
       'current_sit' : situa}
stag = pd.DataFrame(stag)
covariate2 =pd.merge(covariate2, stag, left_on='id', right_on='id')
stag = covariate2.pop('stage')
covariate2.insert(6, 'stage', stag)

In [ ]:
covariate2.to_csv('covariates.csv')

In [ ]:
def apply_function_network(lista_storage,lista_nomi,funzione,directed=False):
    for year in tqdm(range(1990,2019)):
        string='orgorg4_single'+str(year)+'.gpickle'
        network=nx.readwrite.read_gpickle(string)
        storage=str(funzione)+'_orgorg3_single'+str(year)
        lista_storage.append(funzione(network))
        lista_nomi.append(storage)
#         network=nx.readwrite.read_gpickle('bidbid_stage_self_cb'+str(year)+'.gpickle')
#         lista_storage.append(funzione(network))
#         lista_nomi.append(str(funzione)+'_bidder_'+str(year))
    return lista_storage,lista_nomi
def apply_function_network_community(lista_storage,lista_nomi,funzione,directed=False):
    for year in tqdm(range(1999,2020)):
        string='orgorg3_single_/orgorg3_single'+str(year)+'.gpickle'
        network=nx.readwrite.read_gpickle(string)
        storage='Louvain_orgorg3_single'+str(year)
        lista_storage.append(funzione(network))
        lista_nomi.append(storage)
    #network=nx.readwrite.read_gpickle('bidbid_stage_single_cb.gpickle')
    #lista_storage.append(funzione(network))
    #lista_nomi.append(str(funzione)+'_single')
    return lista_storage,lista_nomi

In [ ]:
lista_funzioni_def=[nx.betweenness_centrality,nx.harmonic_centrality,nx.clustering, nx.core_number,nx.eigenvector_centrality, nx.degree,nx.closeness_centrality,nx.pagerank,nx.load_centrality,nx.average_neighbor_degree]



In [ ]:
lista_storage,lista_nomi=apply_function_network(lista_storage,lista_nomi,nx.betweenness_centrality)

In [ ]:
# Array di funzioni da applicare - ciclo for su di esse
#lista_funzioni=[nx.voterank, nx.eigenvector_centrality, nx.degree_centrality, nx.pagerank, nx.closeness_centrality, nx.incremental_closeness_centrality, nx.load_centrality, nx.subgraph_centrality, nx.dispersion, nx.percolation_centrality,nx.average_neighbor_degree,nx.constraint, nx.betweenness_centrality]
# lista_funzioni2=[nx.dispersion,nx.constraint,nx.betweenness_centrality, nx.trophic_levels, nx.harmonic_centrality, nx.enumerate_all_cliques, nx.clustering, nx.core_number, nx.rich_club_coefficient, nx.closeness_vitality]
# lista_funzioni_rifare=[nx.harmonic_centrality, nx.enumerate_all_cliques, nx.clustering,nx.core_number,nx.closeness_vitality]
# Funzione prende df, se direzionato o meno
# lista_funzioni_prova2=[nx.eigenvector_centrality, nx.degree]
lista_storage=[]
lista_nomi=[]
for funzione in lista_funzioni_def:
    try:
        print('Trying'+str(funzione))
        lista_storage2, lista_nomi2=apply_function_network(lista_storage2,lista_nomi2,nx.betweenness_centrality)
    except:
        print(str(funzione)+' did not work')
        continue

In [ ]:
network=nx.readwrite.read_gpickle('orgorg4_single2006.gpickle')

In [ ]:
def apply_function_network(funzione):
    for year in tqdm(range(1990,2021)):
        string2='orgorg4_single'+str(year)+'.gpickle'
        network2=nx.readwrite.read_gpickle(string2)
        storage2=str(funzione)+'_orgorg4_single'+str(year)+'.csv'
        covariate2 = funzione(network2)
        covariate3 = pd.Series(covariate2)
        print('Arrivo qui', year)
        covariate3.to_csv(storage2)
        print('Done', funzione, 'for year', year)

In [ ]:
for function in lista_funzioni_def:
    apply_function_network(function)

In [ ]:
import concurrent
with concurrent.futures.ProcessPoolExecutor() as executor:
    # results = [executor.submit(apply_function_network,funzione) for funzione in lista_funzioni]
    executor.map(apply_function_network, lista_funzioni_def)

In [ ]:
lista_storage2=[]
lista_nomi2=[]
apply_function_network(lista_storage2,lista_nomi2,nx.degree)

In [ ]:
lista_nomi2[10]